In [1]:
import tensorflow as tf
import keras
from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Add
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam
from keras.engine.network import Network
from keras import backend as K

from keras.initializers import glorot_normal,orthogonal

from keras.callbacks import EarlyStopping
import csv

from PIL import Image

%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import cv2
import os
import random

from tqdm import tqdm

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# sess = tf.Session(config=config)
# K.set_session(sess)


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
#どんくらいで学習させるか関連
timesteps=10 #一回のLSTMに入れる値の数
camera=5 #カメラの数

rate = 5 #飛ばすフレーム数（30FPSを30/rateのFPSの動画に疑似変換する．）

#画像関連 
channels=3
img_width=50
img_height=50

#LSTMなどで用いる値
n_hidden = 256 # 出力次元 
epochs = 100 # エポック数 
batch_size = 5 # バッチサイズ

In [3]:
class DataLoader:
    def __init__(self, camera_num, file_name, timesteps, width, height):
        self.camera_num = camera_num
        self.file_name =file_name
        self.samples=0
        self.timesteps=timesteps
        self.width=width
        self.height=height
        self.save_numpy()
        
    def save_numpy(self):
        self.samples = int(self.get_sample()/rate)
        if not (os.path.isfile(self.file_name+"label_"+str(rate)+".npy")):
            self.make_numpy_learnData()
            self.make_numpy_labelData()
        self.learnData=np.load(self.file_name+"learn_"+str(rate)+".npy")
        self.labelData=np.load(self.file_name+"label_"+str(rate)+".npy")
    
    def get_sample(self):
        video_path = self.file_name+"video/0.mp4"
        cap = cv2.VideoCapture(video_path)
        num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        return num
    
    def make_numpy_labelData(self):
        labelData=[]
        csv_selection=csv.reader(open(self.file_name+"video/mintime_optimize.csv", 'r'))
        one_hot = np.eye(self.camera_num)
        for i,row2 in enumerate(csv_selection):
            if(i%rate==0 and i > (self.timesteps-1)*rate-1):
                labelData.append(one_hot[int(row2[1])])
        tmp_list = np.array(labelData)
        tmp_list = tmp_list.astype(np.float)
        np.save(self.file_name+"label_"+str(rate)+".npy", tmp_list)
        print(tmp_list.shape)
    
    
    def make_numpy_learnData(self):
        learnData=[]
        for i in range(0, self.camera_num):
            print()
            img_list=[]
            video_path = self.file_name+"video/"+str(i)+".mp4"
            cap = cv2.VideoCapture(video_path)
            video_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            for j in range(0,video_len):
                
                pro_size=20
                bar = int(j*pro_size/video_len)
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(j / video_len * 100.)
                print('\r{0}/{1} [{2}] {3}%'.format((i+1), self.camera_num, pro_bar, percent), end='')
                
                ret, frame = cap.read()
                if(j%rate==0):
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    img=Image.fromarray(frame)
                    img = img.resize((self.width, self.height))
                    x = np.array(img, dtype=np.float32)
                    x = x / 255.
                    img_list.append(x)
            learnData.append(img_list)
            
        tmp_list=np.array(learnData)
        tmp_list=tmp_list.astype(np.float)
        np.save(self.file_name+"learn_"+str(rate)+".npy", tmp_list)
        print(tmp_list.shape)
    
    
    def make_learndata(self, num=[]):
        data_name=self.file_name+"learn_0_"+str(rate)+".npy"
        if (os.path.isfile(data_name)):
            test=np.load(data_name)
        else:
            all_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
            tmp_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
            for i in range(0, self.labelData.shape[0]):
                #まずはすべての分だけ回す
                pro_size=20
                bar = int(i*pro_size/self.labelData.shape[0])
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(i / self.labelData.shape[0] * 100.)
                print('\r [{0}] {1}%'.format(pro_bar, percent), end='')
                
                count=num[np.argmax(self.labelData[i,:])]
                
                for j in range(0,count):
                    #dataAugumentationする分だけ回す
                    #dataAugmentationは異なるカメラでも同じ時間軸の場合同じ角度で回したいため，はじめに角度を取得する．
                    random_angle = float(random.randint(0,180))
                    
                    for camera in range(0, self.camera_num):
                        #カメラの台数だけ回す
                        #tmp_list[camera]の同じ引数（番号）には角度を同じ分だけずらした同じ時間軸のsequenceが入っている．
                        first_list =np.empty((0, self.width, self.height, 3), np.float)
                        for l in range(0, self.timesteps):
                            #timesteps分のndarrayを作る．
                            tmp_img=self.learnData[camera,i+l]
                            img = self.augumentation(tmp_img, random_angle)
                            first_list =np.append(first_list, [img], axis=0)
                        #あるカメラについて，１つのsequenceがfirst_listに出来上がっている状態．
                        tmp_list[camera]=np.append(tmp_list[camera], [first_list], axis=0)
                if(tmp_list[0].shape[0]>100 or i == self.labelData.shape[0]-1):
                    #処理時間短縮のため，sequenceがtmp_listに溜まってきたら，全体に統合してまたtmp_listを初期化する．
                    for camera in range(0, self.camera_num):
                        all_list[camera]=np.append(all_list[camera], tmp_list[camera], axis=0)
                        tmp_list[camera] = np.empty((0,self.timesteps, self.width, self.height, 3), np.float)
            
            for camera in range(0, self.camera_num):
                print(all_list[camera].shape)
                data_name=self.file_name+"learn_"+str(camera)+"_"+str(rate)
                np.save(data_name, all_list[camera])
                del all_list[camera]
    
    
    def make_labeldata(self, num=[]):
        data_name=self.file_name+"final_label_"+str(rate)+"_.npy"
        if (os.path.isfile(data_name)):
            test=np.load(data_name)
        else:
            test = np.empty((0,5), np.float)
            for i in range(0,self.labelData.shape[0]):
                count=num[np.argmax(self.labelData[i,:])]
                for j in range(0, count):
                    test = np.append(test, [self.labelData[i]], axis=0)
            print(test.shape)
            np.save(data_name, test)
    
    
    
    def get_learndata(self, num):
        data_name=self.file_name+"learn_"+str(num)+"_"+str(rate)+".npy"
        test=np.load(data_name)
        return test
    
    def get_labeldata(self):
        data_name=self.file_name+"final_label_"+str(rate)+"_.npy"
        test=np.load(data_name)
        return test
    
    
    def augumentation(self, img, num):
        height = img.shape[0]                
        width = img.shape[1]                       
        center = (int(width/2), int(height/2))
        angle = num
        scale = 1.0
        trans = cv2.getRotationMatrix2D(center, angle , scale)
        image2 = cv2.warpAffine(img, trans, (width,height))
        return image2
        
    
    def show(self):
        print(0,self.get_learndata(0).shape)
        print(1,self.get_learndata(1).shape)
        print(2,self.get_learndata(2).shape)
        print(3,self.get_learndata(3).shape)
        print(4,self.get_learndata(4).shape)
        print(self.get_labeldata().shape)

In [4]:
data = DataLoader(camera, "./data/20190426/", timesteps, img_width, img_height)
aug_rate=[3,7,7,2,1]
data.make_learndata(aug_rate)
data.make_labeldata(aug_rate)
data.show()

0 (1687, 10, 50, 50, 3)
1 (1687, 10, 50, 50, 3)
2 (1687, 10, 50, 50, 3)
3 (1687, 10, 50, 50, 3)
4 (1687, 10, 50, 50, 3)
(1687, 5)


In [5]:
class Resnet18 :
    ##
    ##　Resnet-18の作成
    ##
    def __init__(self, width, height, channels):
        self.img_rows=height
        self.img_cols=width
        self.img_channels=channels
    
    def rescell(self, data, filters, kernel_size, option=False):
        strides=(1,1)
        if option:
            strides=(2,2)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(data)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)
        
        data=Convolution2D(filters=int(x.shape[3]), kernel_size=(1,1), strides=strides, padding="same")(data)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=(1,1),padding="same")(x)
        x=BatchNormalization()(x)
        x=Add()([x,data])
        x=Activation('relu')(x)
        
        return x
    
    def create_resnet18(self):
        inputs=Input(shape=(self.img_rows, self.img_cols, self.img_channels))
        x=Convolution2D(64,(7,7), padding="same", input_shape=(self.img_rows, self.img_cols),activation="relu")(inputs)
        x=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        
        x=self.rescell(x,128,(3,3),True)
        x=self.rescell(x,128,(3,3))

        x=self.rescell(x,256,(3,3),True)
        x=self.rescell(x,256,(3,3))
        
        x=self.rescell(x,512,(3,3),True)
        x=self.rescell(x,512,(3,3))
        
        model=Model(inputs=inputs,outputs=[x])
        return model

    def get_resnet18(self):
        return self.create_resnet18()

In [6]:
class Prediction :
    def __init__(self, timesteps, n_hidden, n_out, width, height):
        self.maxlen = timesteps
        self.n_hidden = n_hidden
        self.n_out = n_out
        self.width=width
        self.height=height
        self.resnet = Resnet18(width, height, 3).get_resnet18()
        self.sharedLayer = self.create_sharedmodel()
    
    def create_sharedmodel(self):
        inputs = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        x = TimeDistributed(self.resnet,name="resnet")(inputs)
        x = TimeDistributed(GlobalAveragePooling2D(), name="GAP")(x)
        x = TimeDistributed(Dense(512), name="dense")(x)
        predictions = LSTM(self.n_hidden, batch_input_shape = (None, self.maxlen, 35),
             kernel_initializer = glorot_normal(seed=20181020),
             recurrent_initializer = orthogonal(gain=1.0, seed=20181020), 
             dropout = 0.01, 
             recurrent_dropout = 0.01)(x)
        
        shared_layers = Model(inputs, predictions, name="shared_layers")
        
        return shared_layers
    
    def create_model(self):
        model_input1 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input2 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input3 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input4 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input5 = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        mid_feature1 = self.sharedLayer(model_input1)
        mid_feature2 = self.sharedLayer(model_input2)
        mid_feature3 = self.sharedLayer(model_input3)
        mid_feature4 = self.sharedLayer(model_input4)
        mid_feature5 = self.sharedLayer(model_input5)
        
        merged_vector = keras.layers.concatenate([mid_feature1, mid_feature2, mid_feature3, mid_feature4, mid_feature5], axis=-1)
        
        mid_dense = Dense(self.n_hidden, activation='sigmoid')(merged_vector)
        predictions = Dense(self.n_out, activation='sigmoid')(mid_dense)
        
        model = Model(inputs=[model_input1, model_input2, model_input3, model_input4, model_input5], outputs=predictions)
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
        
        return model
    
    def train(self, x_train, t_train, x_vali, t_vali batch_size, epochs) :
        early_stopping = EarlyStopping(patience=20, verbose=1)
        model = self.create_model()
        hist = model.fit([x_train[0], x_train[1], x_train[2], x_train[3], x_train[4]], t_train, batch_size = batch_size, epochs = epochs, 
                         validation_data=([x_vali[0], x_vali[1], x_vali[2], x_vali[3], x_vali[4]], t_vali), shuffle = True, callbacks = [early_stopping], validation_split = 0.1)
        return model, hist

In [7]:
# モデル定義
pred = Prediction(timesteps, n_hidden, camera, img_width, img_height)
model = pred.create_model()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_7 (I

In [8]:
from tqdm import tqdm
test_learn=[np.empty((data.get_labeldata().shape[0],10, 50, 50, 3), np.float)]*camera
vali_learn=[np.empty((0,10, 50, 50, 3), np.float)]*camera
test_label=data.get_labeldata()
vali_label=np.empty((0,camera), np.float)

print(test_label.shape)

for j in range(0,camera):
    test_learn[j]=data.get_learndata(j)

for i in tqdm(range(0,int(0.1*test_label.shape[0]))):
    random_angle = int(random.randint(0,test_label.shape[0]-1))
    for j in range(0,camera):
        vali_learn[j]=np.append(vali_learn[j], [test_learn[j][random_angle]],axis=0)
        test_learn[j]=np.delete(test_learn[j], random_angle,0)
    vali_label=np.append(vali_label, [test_label[random_angle]], axis=0)
    test_label=np.delete(test_label, random_angle,0)



# 学習
model , hist = pred.train(test_learn, test_label, vali_learn, vali_label, batch_size, epochs)
# テスト
score = model.evaluate(test_learn, test_label, batch_size = batch_size, verbose = 1)
print("score:", score)

# -------------------------------------------------------------------------------------
#                            モデルのアーキテクチャの保存
# -------------------------------------------------------------------------------------

# モデルのアーキテクチャの保存①(JSON版)
model_arc_json = model.to_json()
open("model_architecture.json", mode='w').write(model_arc_json)

# -------------------------------------------------------------------------------------
#                            　　モデルの重みの保存
# -------------------------------------------------------------------------------------

# モデルの重みの保存
model.save_weights("weights.hdf5")


Train on 1518 samples, validate on 169 samples
Epoch 1/100
1518/1518 [==============================] - 107s 71ms/step - loss: 0.4749 - acc: 0.7978 - val_loss: 0.5388 - val_acc: 0.7160
Epoch 2/100
1518/1518 [==============================] - 79s 52ms/step - loss: 0.4482 - acc: 0.8084 - val_loss: 0.5315 - val_acc: 0.8000
Epoch 3/100
1518/1518 [==============================] - 77s 50ms/step - loss: 0.4019 - acc: 0.8207 - val_loss: 0.5346 - val_acc: 0.8000
Epoch 4/100
1518/1518 [==============================] - 79s 52ms/step - loss: 0.3165 - acc: 0.8606 - val_loss: 0.6645 - val_acc: 0.8000
Epoch 5/100
1518/1518 [==============================] - 81s 54ms/step - loss: 0.2663 - acc: 0.8813 - val_loss: 0.7362 - val_acc: 0.7160
Epoch 6/100
1518/1518 [==============================] - 81s 53ms/step - loss: 0.2314 - acc: 0.9012 - val_loss: 0.6906 - val_acc: 0.8000
Epoch 7/100
1518/1518 [==============================] - 81s 53ms/step - loss: 0.1935 - acc: 0.9209 - val_loss: 0.9575 - val_acc: 

4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 4
4 4
4 4
4 4
4 4
4 4
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 1
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 3
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 0
4 4


In [ ]:
#試すやつ！

pre = [[0 for i in range(5)] for j in range(5)]
# 正答率集計
preds = model.predict([data.get_learndata(0),data.get_learndata(1),data.get_learndata(2),data.get_learndata(3),data.get_learndata(4)])
correct = 0

f = open("./data/20190426/result.csv", 'w')
writer = csv.writer(f)
for i in range(0,timesteps):
    writer.writerow([0])
        
for i in range(len(preds)):
    pred_result = np.argmax(preds[i,:])
    tar = np.argmax(data.get_labeldata()[i,:])
    for j in range(0,rate):
        writer.writerow([pred_result])
    pre[pred_result][tar]+=1
    print(pred_result, tar)
    if pred_result == tar :
        correct += 1

print("正答率:", 1.0 * correct / len(preds))

In [476]:
a = [0,0,0,0,0]
for i in range(0, 742):
    tar = np.argmax(data.get_labeldata()[i,:])
    a[tar]=a[tar]+1
print(a)

[97, 50, 43, 193, 359]


In [31]:
from tqdm import tqdm
test_learn=[np.empty((data.get_labeldata().shape[0],10, 50, 50, 3), np.float)]*camera
vali_learn=[np.empty((0,10, 50, 50, 3), np.float)]*camera
test_label=data.get_labeldata()
vali_label=np.empty((0,camera), np.float)

print(test_label.shape)

for j in range(0,camera):
    test_learn[j]=data.get_learndata(j)

for i in tqdm(range(0,int(0.1*test_label.shape[0]))):
    random_angle = int(random.randint(0,test_label.shape[0]-1))
    for j in range(0,camera):
        vali_learn[j]=np.append(vali_learn[j], [test_learn[j][random_angle]],axis=0)
        test_learn[j]=np.delete(test_learn[j], random_angle,0)
    vali_label=np.append(vali_label, [test_label[random_angle]], axis=0)
    test_label=np.delete(test_label, random_angle,0)

print(test_learn[0].shape)
print(vali_learn[0].shape)
print(test_label.shape)
print(vali_label.shape)

(1687, 5)



 92%|█████████▏| 155/168 [06:16<00:32,  2.50s/it]


100%|██████████| 168/168 [06:48<00:00,  2.45s/it]

(1519, 10, 50, 50, 3)
(168, 10, 50, 50, 3)
(1519, 5)
(168, 5)
